In [1]:
import nltk
import csv
import math
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from collections import OrderedDict
import operator
from tkinter import *

In [2]:
#Import all Documents
D={}
for x in range(50):
    x=x+1
    file="Documents/"+str(x)+".txt"
    D[x]=open(file).read()
    
    
#appended all documents to allDoc
allDoc=""
for x in range(50):
    x=x+1
    allDoc=allDoc+" \n"+D[x]
    
#Tokens
tokens=nltk.word_tokenize(allDoc)
    
# Unique Tokens 
tokens=list(set(tokens))

#Remove Special characters
removetable=str.maketrans("", "", "'!@#$%^&*()_=-\|][:';:,<.>/?`~")
tokens=[x.translate(removetable) for x in tokens]

#Importing Stoplist
stopWord=open("Stopword-List.txt").read()
stopWord=nltk.word_tokenize(stopWord)

#Decapitalized
tokens=[element.lower() for element in tokens]

# Removing StopWords
tokens=[x for x in tokens if x.isalnum() and x not in stopWord]

tokens=[element.lower() for element in tokens]

#Sorted Tokens
tokens=sorted(tokens)
print("size: ",len(tokens))

#Document wise Tokenization
docToken={}
for x in range(1,50):
    docToken[x]=nltk.word_tokenize(D[x])
    
    
#Remove Special characters Doc Wise
removetable=str.maketrans("", "", "'!@#$%^&*()_=-\|][:';:,<.>/?`~")
for x in range(1,50):
    docToken[x]=[y.translate(removetable) for y in docToken[x]]
    
#doc wise sorted
for x in range(1,50):
    docToken[x]=sorted(docToken[x])
    
#Decaptilized Doc Wise
for x in range(1,50):
    docToken[x]=[element.lower() for element in docToken[x]]
    
for x in range(1,50):
    docToken[x]=[y for y in docToken[x] if y.isalnum() and y not in stopWord]

size:  12031


In [3]:
docV={}

for x in range(1,50):
    docV[x]=dict.fromkeys(tokens,0) 
    
#Term Frequency in  a document
for x in range(1,50):
    for word in docToken[x]:
        docV[x][word]+=1    
        
#tf
tfDocV={}
for x in range(1,50):
    tfDocV[x]={}
    for word,count in docV[x].items():
        tfDocV[x][word]=count
        
#unique Token Doc wise
for x in range(1,50):
    docToken[x]=set(docToken[x])
    docToken[x]=list(set(docToken[x]))
    

wordDcount=dict.fromkeys(tokens,0)
for word in tokens:
    for x in range(1,50):
        if word in docToken[x]:
            wordDcount[word]+=1
            
#idf            
idfDict = {}
for word in tokens:
    if wordDcount[word]>0:
        count=wordDcount[word]
        if count>50:
            count=50
    idfDict[word]=math.log(50/count)
    
#tf-idf    
tfidf={}
for x in range(1,50):
    tfidf[x]={}
    for word in docV[x]:
        tfidf[x][word]=tfDocV[x][word]*idfDict[word]
        

In [4]:
def show_entry_fields():
    query=e1.get()
    
    qt=nltk.word_tokenize(query)
    
    #Remove Special characters
    removetable=str.maketrans("", "", "'!@#$%^&*()_=-\|][:';:,<.>/?`~")
    qt=[x.translate(removetable) for x in qt]
    
    #Decapitalized
    qt=[element.lower() for element in qt]
    
    qt=[y for y in qt if y.isalnum() and y not in stopWord]
    qt=list(set(qt))
    
    qtV=dict.fromkeys(tokens,0)
    for word in qt:
        try:
            qtV[word]+=1
        except KeyError:
            None
        
    #quert idf
    for words in qtV:
        try:
            qtV[words]=qtV[words]*idfDict[word]
        except KeyError:
            None
        
    #cosine Similarity
    res={}
    temp=0
    vec1=np.array([list(qtV.values())])
    for x in range(1,50):
        vec2=np.array([list(tfidf[x].values())])
        if cosine_similarity(vec1,vec2)>0:
            temp=cosine_similarity(vec1,vec2)[0][0]
            res[x]=temp
            
    res=sorted(res.items(), key=operator.itemgetter(1), reverse=True)
    lst={}
    for items in res:
        if items[1]>=0.005:
            lst[items[0]]=items[1]
    
    
    #-----------------------------
    print(len(res)-len(lst))
    print(lst)
    lst1=list(lst.keys())
   
    lst2=list(lst.values())
    lst2=["%.5f" % v for v in lst2]
    
    e2.delete(0,END)
    e3.delete(0,END)
    e4.delete(0,END)

    e2.insert(15,lst1)
    e3.insert(15,lst2)
    e4.insert(15,len(res)-len(lst))        

In [6]:
master = Tk()
master.geometry('900x300')
master.title("Vector Space Model")

Label(master, text="Enter Query: ",width=20,font=("bold", 10),padx=10, pady=20).grid(row=0)
Label(master, text="Result (Document IDs (1st,2nd,3rd,...))",width=30,font=("bold", 10),padx=10, pady=20).grid(row=1)
Label(master, text="Tf-IDF Rank",width=20,font=("bold", 10),padx=10, pady=20).grid(row=2)
Label(master, text="Document Trimed (alpha=0.005)",width=20,font=("bold", 10),padx=10, pady=20).grid(row=3)

e1 = Entry(master,width=100)
e2 = Entry(master,width=100)
e3 = Entry(master,width=100)
e4 = Entry(master,width=100)

e1.insert(15,"crowd busy")

e1.grid(row=0, column=1)
e2.grid(row=1, column=1)
e3.grid(row=2, column=1)
e4.grid(row=3, column=1)

Button(master, text='Search', command=show_entry_fields).grid(row=5, column=1, sticky=W, pady=4)

mainloop( )

10
{30: 0.3470628073369998, 14: 0.03475912671051275, 9: 0.029206253627892916, 23: 0.028157900419365476, 49: 0.014439353321322306, 34: 0.011353497518898964, 13: 0.008639535903065905, 16: 0.007456668012340061, 36: 0.0072017534017403285, 46: 0.007108177905769288, 40: 0.006568300404349771, 6: 0.0058733594495580955}
